In [2]:
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('mnist_data/', download=True, train=True, transform=transform, target_transform=None)
testset = datasets.MNIST('mnist_data/', download=True, train=False, transform=transform, target_transform=None)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=False)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

class Model(nn.Module):
    def __init__(self,imgWidth,imgHeight,numClasses):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(imgWidth*imgWidth, 20)  
        self.fc2 = nn.Linear(20,numClasses)
        self.softmax = nn.Softmax()

    def forward(self, img):
        img = img.view(img.size(0), -1)  
        outputOfLayer1 = self.fc1(img)
        finalOutput = self.softmax(self.fc2(outputOfLayer1))
        return finalOutput
    
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


if __name__ == "__main__":
    imgWidth = 28
    imgHeight = 28
    numClasses = 10

    model = Model(imgWidth, imgHeight, numClasses).to(device='cpu')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)

    numEpochs = 20
    for epoch in range(numEpochs):
        running_loss = 0.0
        for dataSample in trainloader:
            image, label = dataSample
            optimizer.zero_grad()  
            output = model(image)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")

    print('Training Finished')

    test_accuracy = calculate_accuracy(model, testloader)

    print(f"Test Accuracy: {100 * test_accuracy:.2f}%")

Epoch 1, Loss: 1.6833977451456634
Epoch 2, Loss: 1.5630304072457335
Epoch 3, Loss: 1.554288390348715
Epoch 4, Loss: 1.5488917417109394
Epoch 5, Loss: 1.5465360127532406
Epoch 6, Loss: 1.54400803806431
Epoch 7, Loss: 1.5415971790041243
Epoch 8, Loss: 1.5405969810384168
Epoch 9, Loss: 1.5391353860592791
Epoch 10, Loss: 1.5383954254040586
Epoch 11, Loss: 1.5369901953221383
Epoch 12, Loss: 1.5362782132650998
Epoch 13, Loss: 1.5364553313265477
Epoch 14, Loss: 1.5364158186577022
Epoch 15, Loss: 1.5348231976728703
Epoch 16, Loss: 1.5338575332912046
Epoch 17, Loss: 1.5340368967574796
Epoch 18, Loss: 1.5350324325978375
Epoch 19, Loss: 1.5331055796476825
Epoch 20, Loss: 1.5335976622506244
Training Finished
Test Accuracy: 92.06%
